# CS 634 Midterm Project              By Chike Egbuchulam(coe5)

## Setup 

First you want to import the libraries essential to run the the FP and Apriori Algorithms

In [ ]:
import pandas as pd
import numpy as np
import mlxtend as ml
from bfassociationrules import assocRules

Now you must decide what Bakery you want to go to